In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
# train_test_split: Para dividir el dataset en conjuntos de entrenamiento y validación.
# Uso: Asegura que el modelo se evalúe en datos no vistos durante el entrenamiento.
# GridSearchCV: Para la optimización de hiperparámetros de nuestro modelo y/o pipeline.
# Uso: Busca sistemáticamente la mejor combinación de parámetros.

from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
# StandardScaler: Para estandarizar (escalar) las características numéricas.
# Uso: Transforma los datos para que tengan media 0 y desviación estándar 1, útil para muchos algoritmos.
# OneHotEncoder: Para convertir características categóricas (como 'Sex', 'Embarked') a formato numérico binario.
# Uso: Permite que los modelos de ML trabajen con datos categóricos.
# FunctionTransformer: Para integrar funciones personalizadas (como tus funciones de ingeniería de características)
# Uso: Te permite usar tus funciones Python dentro del pipeline de Scikit-learn.

from sklearn.impute import SimpleImputer
# Uso: Para manejar los valores faltantes. Por ejemplo, rellenar 'Age' o 'Fare' con la media.

from sklearn.compose import ColumnTransformer
# Uso: Permite aplicar diferentes transformaciones a diferentes columnas del DataFrame.
#      Es clave para nuestro pipeline, ya que tenemos columnas numéricas y categóricas.

from sklearn.pipeline import Pipeline
# Uso: La clase principal para construir tu pipeline. Encadena todos los pasos de preprocesamiento y el modelo.
#      Esto garantiza que todas las transformaciones se apliquen consistentemente.

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier 
# Uso: El algoritmo de Machine Learning que usaremos para nuestro modelo de clasificación binaria.
#      Es un clasificador de conjunto robusto y de alto rendimiento.

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# Uso: Para evaluar el rendimiento de nuestro modelo.
# accuracy_score: La métrica principal para el concurso de Kaggle.
# confusion_matrix: Muestra la cantidad de verdaderos positivos, negativos, falsos positivos y negativos.
# classification_report: Proporciona precisión (precision), recall (sensibilidad) y F1-score por clase.

# Librerías para visualización de datos (aunque en este notebook será más para mostrar resultados)
import matplotlib.pyplot as plt
import seaborn as sns
# Uso: Para crear gráficos, como la matriz de confusión, para visualizar y entender los resultados del modelo.
import joblib as jl

# Configuración para gráficos
sns.set_style("whitegrid")
plt.style.use("seaborn-v0_8-darkgrid")
import numpy as np
# Uso: Ajustes estéticos para que los gráficos se vean bien.

In [2]:
sns.set_style('whitegrid')
plt.style.use('seaborn-v0_8-darkgrid')

In [3]:
df_train_raw = pd.read_csv('../data/raw/train.csv')
df_test_raw = pd.read_csv('../data/raw/test.csv')
df_join_raw = pd.concat([df_train_raw.drop('Survived', axis=1), df_test_raw], ignore_index=True)
temp_df = df_join_raw.copy()
print(f'Dimensiones iniciales: {df_join_raw.shape}')
print(f'Columnas iniciales: {list(df_join_raw.columns)}')

Dimensiones iniciales: (1309, 11)
Columnas iniciales: ['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']


In [4]:

def get_title_from_name(df):
    df = df.copy()
    def extract_title(name):
        title_extracted = re.search(' ([A-Za-z]+)\\.', name)
        if title_extracted:
            return title_extracted.group(1)
        return 'rare'
    df['Title'] = df['Name'].apply(extract_title)
    df = df.drop('Name', axis=1)
    return df

def classify_titles(df):
    df = df.copy()
    def classify(title):
        if title in ('Countess', 'Lady', 'Jonkheer'):#3
            return 3
        elif title in ('Miss', 'Mrs', 'Mme', 'Mlle', 'Ms', 'Master', 'Dona'):#7
            return 2
        elif title in ('Dr', 'Rev', 'Col', 'Capt', 'Sir', 'Major', 'Don'):#7
            return 1
        return 0 #('Mr', 'rare')
    df['Title'] = df['Title'].apply(classify)
    return df


def get_deck_from_cabin(df):
    df = df.copy()
    df['Cabin'] = df['Cabin'].fillna('Unknown')
    df['Deck'] = df['Cabin'].transform(lambda x: x[0])
    df = df.drop('Cabin', axis=1)
    return df

def get_agency_ticket_numbers_from_ticket(df):
    df = df.copy()
    def extract_prefix(ticket):
        prefix_extracted = re.match(r'([A-Za-z\./]+)', ticket)
        if prefix_extracted:
            return prefix_extracted.group(1).replace('.', '').replace('/', '').upper()
        return 'NO_AGENCY'
    df['Agency'] = df['Ticket'].apply(extract_prefix)
    ticket_counts = df['Ticket'].value_counts()
    df['TicketNumber'] = df['Ticket'].map(ticket_counts)
    df = df.drop('Ticket', axis=1)
    return df

def familysize_isAlone_from_sibsp_parch(df):
    df = df.copy()
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
    df['IsAlone'] = (df['FamilySize'] == 1).astype(int)
    df = df.drop(['Parch', 'SibSp', 'PassengerId'], axis=1)
    return df

def reorgize_ticket_class(df):
    df = df.copy()
    def condition_class(ticket_class):
        class_mapping = { 1: 2,  2: 1, 3: 0}
        return class_mapping.get(ticket_class, 0)
    df['Pclass'] = df['Pclass'].apply(condition_class)
    return df

def age_imputed_from_title_pclass(df):
    df = df.copy()
    df['Age'] = df.groupby(['Pclass', 'Title'])['Age'].transform(lambda x: x.fillna(x.median()))
    df['Age'] = temp_df['Age'].fillna(temp_df['Age'].median())
    return df

feature_engineering_pipeline = Pipeline([
    ('get_title', FunctionTransformer(get_title_from_name, validate=False)),
    ('classify_title', FunctionTransformer(classify_titles, validate=False)),
    ('get_deck', FunctionTransformer(get_deck_from_cabin, validate=False)),
    ('get_agency', FunctionTransformer(get_agency_ticket_numbers_from_ticket, validate=False)),
    ('familySize', FunctionTransformer(familysize_isAlone_from_sibsp_parch, validate=False)),
    ('reorganize_pclass', FunctionTransformer(reorgize_ticket_class, validate=False)),
    ('imputer_custom_age', FunctionTransformer(age_imputed_from_title_pclass, validate=False))
])
feature_engineering_pipeline.set_output(transform='pandas')
print('Cargado transformadores de ingenieria de features')

Cargado transformadores de ingenieria de features


In [5]:
numeric_features = ['Age', 'Fare', 'TicketNumber', 'FamilySize', 'Pclass', 'Title']
numeric_transformer = Pipeline(steps=[
    ('inputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_features = ['Embarked', 'Deck', 'Agency', 'Sex']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', numeric_transformer, numeric_features),
        ('categorical', categorical_transformer, categorical_features)
    ],
    remainder='drop'
)

preprocessor.set_output(transform='pandas')
print('preprocessor created')

preprocessor created


In [6]:
full_pipeline = Pipeline(steps=[
    ('features_engineering', feature_engineering_pipeline),
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])
print('defined full pipelines')

defined full pipelines


In [7]:
X = df_train_raw.drop(['Survived'], axis=1)
y = df_train_raw['Survived']
X_test_final = df_test_raw.copy()
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)

In [8]:
param_grid = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [10, 20, None],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4]
}
print("Cuadrícula de parámetros definida.")

Cuadrícula de parámetros definida.


In [9]:
#Entrenar el modelo
full_pipeline.fit(X_train, y_train)
print('Modelo entrenado!!')

#Probar diferentes hiperparámetros
grid_search = GridSearchCV(estimator=full_pipeline, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
search_model = grid_search.fit(X_train, y_train)
best_model = search_model.best_estimator_
print(f'Mejor modelo encontrado: {best_model}')
print('Grid Search completado!')


Modelo entrenado!!
Fitting 5 folds for each of 81 candidates, totalling 405 fits
Mejor modelo encontrado: Pipeline(steps=[('features_engineering',
                 Pipeline(steps=[('get_title',
                                  FunctionTransformer(func=<function get_title_from_name at 0x000002CA7AB26160>)),
                                 ('classify_title',
                                  FunctionTransformer(func=<function classify_titles at 0x000002CA7AB24860>)),
                                 ('get_deck',
                                  FunctionTransformer(func=<function get_deck_from_cabin at 0x000002CA2D76F240>)),
                                 ('get_agency',
                                  FunctionTr...
                                                  ['Age', 'Fare',
                                                   'TicketNumber', 'FamilySize',
                                                   'Pclass', 'Title']),
                                                 ('c

In [10]:
""" y_pred_val = full_pipeline.predict(X_val) """
y_pred_val = best_model.predict(X_val)
print('Predicciones realizadas en el conjunto de validación!')
res_accuracy = accuracy_score(y_val, y_pred_val)
print(f'Accuracy del modelo: {res_accuracy:.4f}')
print('--' * 20)
print('Confussion Matrix:')
print(confusion_matrix(y_val, y_pred_val))
print('--' * 20)
print('response de clasificacion:')
print(classification_report(y_val, y_pred_val))
print('--' * 20)

Predicciones realizadas en el conjunto de validación!
Accuracy del modelo: 0.8045
----------------------------------------
Confussion Matrix:
[[97 13]
 [22 47]]
----------------------------------------
response de clasificacion:
              precision    recall  f1-score   support

           0       0.82      0.88      0.85       110
           1       0.78      0.68      0.73        69

    accuracy                           0.80       179
   macro avg       0.80      0.78      0.79       179
weighted avg       0.80      0.80      0.80       179

----------------------------------------


c:\Users\Albeiro Lozano\Desktop\PROJECTS\titanic-survival\titanic-survival\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [11]:
res = best_model.predict(df_test_raw)
subimission = pd.DataFrame({
    'PassengerId': df_test_raw['PassengerId'],
    'Survived': res
})
subimission.to_csv('../data/result/submission.csv', index=False)
print('Submisión guardada en ../data/processed/submission.csv')

Submisión guardada en ../data/processed/submission.csv


c:\Users\Albeiro Lozano\Desktop\PROJECTS\titanic-survival\titanic-survival\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [12]:
jl.dump(best_model, open('../models/best_model.pkl', 'wb'))
print('Modelo guardado!!')

Modelo guardado!!
